# Building a tweet classifier using Transformers
This notebooks contains the experiments and details to building a pipeline that classifies tweets

In [5]:
from datasets import load_dataset
emotions = load_dataset('emotion')
ds_train = emotions['train']

In [6]:
print(emotions)
print('__'*40)
print(f'''Column names: {ds_train.column_names}
Number of samples in training data: {len(ds_train)}'''
)
for index in range(5):
    sample = ds_train[index]
    print(f'''
sample label: {sample['label']}
sample text: {sample['text']}''')
print('__'*40)
print(ds_train.features)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})
________________________________________________________________________________
Column names: ['text', 'label']
Number of samples in training data: 16000

sample label: 0
sample text: i didnt feel humiliated

sample label: 0
sample text: i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake

sample label: 3
sample text: im grabbing a minute to post i feel greedy wrong

sample label: 2
sample text: i am ever feeling nostalgic about the fireplace i will know that it is still on the property

sample label: 3
sample text: i am feeling grouchy
________________________________________________________________________________
{'text': Value(dtype='string', i

In [7]:
for label, text in zip(ds_train['label'][:5], ds_train['text'][:5]):
    print(label, text)

0 i didnt feel humiliated
0 i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake
3 im grabbing a minute to post i feel greedy wrong
2 i am ever feeling nostalgic about the fireplace i will know that it is still on the property
3 i am feeling grouchy


# Datasets to DataFrames

In [8]:
import pandas as pd
emotions.set_format(type='pandas')
df_emotions = emotions['train'][:]
df_emotions.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


What is **int2str**?
- It's a mapping function that maps a numerical labels to their string counter
part.
>  int2str function is typically used to convert integer sequences back into string sequences. It is commonly used in natural language processing tasks, such as machine translation or text generation.

In [9]:
# a function that takes a row's label and maps it to it's corresponding string
def map_label(row_label):
    return emotions['train'].features['label'].int2str(row_label)

In [10]:
df_emotions['label_name'] = df_emotions['label'].apply(map_label)
df_emotions.head()

,text,label,label_name
0,i didnt feel humiliated,0,sadness
1,i can go from feeling so hopeless to so damned...,0,sadness
2,im grabbing a minute to post i feel greedy wrong,3,anger
3,i am ever feeling nostalgic about the fireplac...,2,love
4,i am feeling grouchy,3,anger


## Visualizing data for better understanding

In [11]:
import matplotlib.pyplot as plt

In [12]:
df_emotions['label_name'].value_counts(ascending=True).plot.barh()
plt.title('Class Frequencies')
plt.ylabel('Class type')
plt.tight_layout()
plt.show();

<Figure size 640x480 with 1 Axes>

From the plot above we can observe that the classes `joy` and `sadness` apear more
frequently, where as `love` and `surprise` are less frequent. This problem is commonly known as 
class imbalance and there are multiple ways to address it:
- Randomly oversample the minority class
- Randomly undersample the majority class
- Gather more labeled data from underrepresented classes.

There is a great sklearn library that addresses this issue [imbalanced-learn](https://imbalanced-learn.org/stable/).

**Converting text to tokens**
- Models usually cannot receive raw text as input instead they assume that text has been
tokenized and and encoded as numerical vectors

The different ways of tokenizing text are character, word and subword with the most effective being sub word tokenization..

In [13]:
from transformers import AutoTokenizer
model_checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [14]:
text = 'tokenization is a core task in NLP'
encoded_text = tokenizer(text)
print(encoded_text)

{'input_ids': [101, 19204, 3989, 2003, 1037, 4563, 4708, 1999, 17953, 2361, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [15]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

['[CLS]', 'token', '##ization', 'is', 'a', 'core', 'task', 'in', 'nl', '##p', '[SEP]']


In [ ]:
from transformers import AutoTokenizer
model_checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

text = 'tokenization is a core task in NLP'
encoded_text = tokenizer(text)
print(encoded_text)

tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

In [16]:
# tokenizing the whole dataset
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
    # padding = True, this pads sequences in the same batch to the size of the longest one in the patch.
    # truncation = True, conversely this truncates sequences that are long than tokenize.model_max_length

In [17]:
emotions.set_format(None)
print(tokenize(emotions['train'][:2]))

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


# Using Transformers as feature extractors

In [18]:
from transformers import AutoModel
import torch
model_checkpoint = 'distilbert-base-uncased'
device_check = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_check)
print(f'current device is {device_check}')
model = AutoModel.from_pretrained(model_checkpoint).to(device)

current device is cuda


In [19]:
# getting the last hidden states
text = 'this is a text' # here n_tokens = 4 tokens + 2 special tokens
inputs = tokenizer(text, return_tensors='pt') # shape=[batch_size, n_tokens]
print(f'input tensor shape {inputs["input_ids"].size()}')
print(inputs)


input tensor shape torch.Size([1, 6])
{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3793,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [20]:
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad(): # context manager
    outputs = model(**inputs)
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.2535, -0.1091,  0.0759,  ..., -0.1054,  0.2257,  0.4615],
         [-0.7154, -0.2742, -0.0954,  ..., -0.3597,  0.5174,  0.3450],
         [-0.4100, -0.2195,  0.1850,  ..., -0.0136,  0.1607,  1.0422],
         [-0.2523, -0.0254,  0.2472,  ..., -0.1637,  0.0865,  1.0516],
         [-0.0570, -0.1057,  0.0662,  ...,  0.0584,  0.1034,  0.2971],
         [ 0.9184,  0.1577, -0.4105,  ...,  0.1509, -0.7327, -0.1572]]],
       device='cuda:0'), hidden_states=None, attentions=None)


In [21]:
outputs.last_hidden_state.size()

torch.Size([1, 6, 768])

Looking at the hidden state tensor we see that it has the shape `[batch_size, n_tokens, hidden_dim]`

In [22]:
def extract_hidden_states(batch):
    inputs = {k:v.to(device) for k,v in batch.items()
               if k in tokenizer.model_input_names}
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    return {"hidden_state": last_hidden_state[:, 0].cpu().numpy()}

In [23]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

In [24]:
print(emotions_encoded["train"].column_names)
print(emotions_encoded['train'][:4]['input_ids'])

['text', 'label', 'input_ids', 'attention_mask']
[[101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 10047, 9775, 1037, 3371, 2000, 2695, 1045, 2514, 20505, 3308, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2572, 2412, 3110, 16839

In [25]:
emotions_encoded.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [26]:
emotions_hidden = emotions_encoded.map(extract_hidden_states, batched=True)
emotions_hidden['train'].column_names

['text', 'label', 'input_ids', 'attention_mask', 'hidden_state']

In [27]:
import numpy as np
x_train = np.array(emotions_hidden['train']['hidden_state'])
x_validation = np.array(emotions_hidden['validation']['hidden_state'])
y_train = np.array(emotions_hidden['train']['label'])
y_validation = np.array(emotions_hidden['validation']['label'])
x_train.shape, x_validation.shape

((16000, 768), (2000, 768))

In [28]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(max_iter=3000)
lr_clf.fit(x_train, y_train)
lr_clf.score(x_validation, y_validation)

0.6335

In [29]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(x_train, y_train)
dummy_clf.score(x_validation, y_validation)

0.352

In [30]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize='true')
    fig, ax = plt.subplots(figsize=(6,6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap='Blues', values_format='.2f', ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()
y_preds = lr_clf.predict(x_validation)
labels = emotions['train'].features['label'].names
plot_confusion_matrix(y_preds, y_validation, labels)


<Figure size 600x600 with 1 Axes>

# Fine-tuning a transformer for text classification

In [31]:
from transformers import AutoModelForSequenceClassification
number_of_labels = 6
model = AutoModelForSequenceClassification \
        .from_pretrained(model_checkpoint, num_labels=number_of_labels) \
        .to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'f1_score': f1,
        'accuracy': acc,
    }


In [33]:

from transformers import TrainingArguments, Trainer
batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_checkpoint}-finetuned-emotion"
training_args = TrainingArguments(
    output_dir=f'./model/{model_name}',
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=False,
    log_level='error',
)

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=emotions_encoded['train'],
    eval_dataset=emotions_encoded['validation'],
    tokenizer=tokenizer,
)
trainer.train()

  0%|          | 0/500 [00:00<?, ?it/s]

{'loss': 0.842, 'learning_rate': 1e-05, 'epoch': 1.0}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.32205256819725037, 'eval_f1_score': 0.902780995244089, 'eval_accuracy': 0.904, 'eval_runtime': 1.8447, 'eval_samples_per_second': 1084.176, 'eval_steps_per_second': 17.347, 'epoch': 1.0}
{'loss': 0.254, 'learning_rate': 0.0, 'epoch': 2.0}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.21328745782375336, 'eval_f1_score': 0.9246969472920823, 'eval_accuracy': 0.925, 'eval_runtime': 2.2908, 'eval_samples_per_second': 873.069, 'eval_steps_per_second': 13.969, 'epoch': 2.0}
{'train_runtime': 127.6898, 'train_samples_per_second': 250.607, 'train_steps_per_second': 3.916, 'train_loss': 0.5480169143676757, 'epoch': 2.0}


TrainOutput(global_step=500, training_loss=0.5480169143676757, metrics={'train_runtime': 127.6898, 'train_samples_per_second': 250.607, 'train_steps_per_second': 3.916, 'train_loss': 0.5480169143676757, 'epoch': 2.0})

In [38]:
emotions_encoded['test'][1]

{'label': tensor(0),
 'input_ids': tensor([  101, 10047,  2039, 16616,  2026,  9927,  2138,  1045,  2514, 28543,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [35]:
preds_output = trainer.predict(emotions_encoded['validation'])
preds_output.metrics

  0%|          | 0/32 [00:00<?, ?it/s]

{'test_loss': 0.21328745782375336,
 'test_f1_score': 0.9246969472920823,
 'test_accuracy': 0.925,
 'test_runtime': 2.7189,
 'test_samples_per_second': 735.599,
 'test_steps_per_second': 11.77}

# Creating a Gradio interface

In [67]:
from transformers import TextClassificationPipeline
import gradio as gr
model = trainer.model
pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True,
    device='cuda'
)

def pred(text):
    pred = pipe([text])
    preds = [dictionary['score'] for dictionary in pred[0]]
    return map_label(preds.index(max(preds)))

gr.Interface(
    fn=pred,
    inputs='text',
    outputs='text',
).launch()
print(emotions['train'].features['label'])

d:\codes\natural_language_processing\.venv\Lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)


d:\codes\natural_language_processing\.venv\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
